In [141]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [142]:
import os
import traci
import sys
import argparse
import tensorflow as tf
import numpy as np
from dotenv import load_dotenv

# Import internal modules
from rl_package.rl_logic.Environnement import EnvironnementSumo
from rl_package.rl_logic.Agent import AgentSumo
from rl_package.params import *


In [143]:


# Load environment variables
load_dotenv()


#SUMO_BIN = r"C:/Program Files/rl_project/Eclipse/Sumo/bin/sumo.exe"
SIMUL_CONFIG = r"double_traffic/double_traffic.sumo.cfg"


WINDOW=2000
BATCH_SIZE=6
# SUMO command


In [144]:

def preprocess():
    """
    Determines the number of inputs and outputs required for the model.
    """
    sumoCmd = [SUMO_BIN, "-c", SIMUL_CONFIG, '--start', '--no-warnings']
    env = EnvironnementSumo(sumoCmd, WINDOW)
    inputs_per_agents = []
    outputs_per_agents = []
    for trafficlight in env.trafficlights_ids:

    # Get the number of lanes that are not intersections
        n_lanes = len(env.control_lanes(trafficlight))
        inputs_per_agents.append(n_lanes*2)

        # Get the number of valid traffic light phases (excluding yellow phases)
        n_outputs = len(env.get_phase_without_yellow(trafficlight)[0])
        outputs_per_agents.append(n_outputs)

    # Get the number of agents

    env.close()
    return inputs_per_agents, outputs_per_agents  # Inputs: lane states (queue + vehicle count), Outputs: traffic light phases


In [145]:
preprocess()

 Retrying in 1 seconds


Step #0.00 (0ms ?*RT. ?UPS, TraCI: 1ms, vehicles TOT 0 ACT 0 BUF 0)                      


([64, 64], [4, 4])

In [ ]:

def train_models(inputs_per_agents, outputs_per_agents, type_model="DQN"):
    """
    Trains a reinforcement learning model to optimize traffic lights.
    Saves the trained model after completion.
    """

    agents = [AgentSumo(type_model, inputs, outputs) for inputs,outputs  in zip(inputs_per_agents,outputs_per_agents)]
    for agent in agents:
        agent.build_model()
        # model_path = f"models/{type_model}.keras"
        # if os.path.exists(model_path):
        #     print(f"🔄 Loading pre-trained model {type_model}...")
        #     agents[i].model_action=tf.keras.models.load_model(model_path)
        #     agents[i].model_target=tf.keras.models.load_model(model_path)
        #print('fvvgfv', agent.n_inputs, agent.n_outputs)
    sumoCmd = [SUMO_BIN, "-c", SIMUL_CONFIG, '--start', '--no-warnings']

    for episode in range(EPISODE):
        print(f'🔄 Episode {episode}/{EPISODE}')
        env = EnvironnementSumo(sumoCmd, WINDOW)
        #epsilon = max(1 - episode / EPISODE, 0.01)  # Decaying epsilon for exploration
        epsilon=0.2
        # récupère le nom des agents
        traffic_lights = env.trafficlights_ids



        # appel une fonction et récupère une partie du dictionnaire
        states = [env.get_states_per_traffic_light(traffic_light) for traffic_light in traffic_lights]

        for _ in range(50):  # Steps per episode
            actions = [agent.epsilon_greedy_policy(np.array(states[i]),epsilon) for i, agent in enumerate(agents)]
            #print('ici')
            next_states, rewards = env.step(actions)
            for i in range(len(agents)):
                agents[i].add_to_memory(np.array(states[i]), np.array(actions[i]), np.array(rewards[i]), np.array(next_states[i]))
            states = next_states
            # Train the model if there is enough experience in memory

            if len(agents[0].replay_buffer) >= BATCH_SIZE * 1:
                for agent in agents:
                    agent.training_step(BATCH_SIZE)

            # Stop the simulation if there are no vehicles left
            if env.get_total_number_vehicles() == 0:
                break

        # Update target network every 5 episodes for Double/Dueling DQN
        if episode % 5 == 0 and type_model != 'DQN':
            for agent in agents:
                agent.model_target.set_weights(agent.model_action.get_weights())

        env.close()
    return agents

    # Save the trained model
    # model_path = f"models/{type_model}.keras"
    # agents[i].model_action.save(model_path)
    # print(f"✅ Model saved at: {model_path}")


In [147]:

def scenario(agents):
    """
    Runs a SUMO simulation using the trained agent.
    """
    sumoCmd = [SUMO_GUI_BIN, "-c", SIMUL_CONFIG, '--start', '--no-warnings']
    env = EnvironnementSumo(sumoCmd, WINDOW)
    env.full_simul(agents)


In [148]:

type_model = "2DQN"
inputs_per_agents, outputs_per_agents = preprocess()
agents  = train_models(inputs_per_agents, outputs_per_agents,type_model)


 Retrying in 1 seconds


Step #0.00 (0ms ?*RT. ?UPS, TraCI: 1ms, vehicles TOT 0 ACT 0 BUF 0)                      
🚀 Création d'un nouveau modèle 2DQN...
🚀 Création d'un nouveau modèle 2DQN...
🔄 Episode 0/20
 Retrying in 1 seconds


Step #1000.00 (0ms ?*RT. ?UPS, TraCI: 169ms, vehicles TOT 835 ACT 174 BUF 356)              ?*RT. ?UPS, TraCI: 208ms, vehicles TOT 666 ACT 133 BUF 198)             
🔄 Episode 1/20
 Retrying in 1 seconds


Step #1000.00 (0ms ?*RT. ?UPS, TraCI: 161ms, vehicles TOT 922 ACT 97 BUF 269)               ?*RT. ?UPS, TraCI: 191ms, vehicles TOT 688 ACT 111 BUF 176)             
🔄 Episode 2/20
 Retrying in 1 seconds


Step #1000.00 (0ms ?*RT. ?UPS, TraCI: 211ms, vehicles TOT 970 ACT 92 BUF 221)               ~= 10.00*RT, ~88000.00UPS, TraCI: 279ms, vehicles TOT 736 ACT 88 BUF 128
🔄 Episode 3/20
 Retrying in 1 seconds


...

Step #1000.00 (0ms ?*RT. ?UPS, TraCI: 169ms, vehicles TOT 988 ACT 85 BUF 203)               ~= 10.00*RT, ~99000.00UPS, TraCI: 202ms, vehicles TOT 733 ACT 99 BUF 131
🔄 Episode 16/20
 Retrying in 1 seconds


Step #1000.00 (1ms ~= 10.00*RT, ~101000.00UPS, TraCI: 165ms, vehicles TOT 993 ACT 101 BUF 1 ?*RT. ?UPS, TraCI: 188ms, vehicles TOT 750 ACT 104 BUF 114)             
🔄 Episode 17/20
 Retrying in 1 seconds


Step #1000.00 (0ms ?*RT. ?UPS, TraCI: 178ms, vehicles TOT 943 ACT 81 BUF 248)               ?*RT. ?UPS, TraCI: 229ms, vehicles TOT 702 ACT 102 BUF 162)             
🔄 Episode 18/20
 Retrying in 1 seconds


Step #1000.00 (0ms ?*RT. ?UPS, TraCI: 181ms, vehicles TOT 958 ACT 165 BUF 233)              ~= 10.00*RT, ~141000.00UPS, TraCI: 217ms, vehicles TOT 721 ACT 141 BUF 1
🔄 Episode 19/20
 Retrying in 1 seconds


Step #1000.00 (0ms ?*RT. ?UPS, TraCI: 181ms, vehicles TOT 1028 ACT 93 BUF 163)              ~= 10.00*RT, ~86000.00UPS, TraCI: 252ms, vehicles TOT 762 ACT 86 BUF 102


In [150]:
scenario(agents)

 Retrying in 1 seconds


FatalTraCIError: Connection closed by SUMO.